# Processing the final dataset



In [1]:
import ee
import geemap

# Authenticate to Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

data_folder = "projects/amazon-forest-regrowth/assets"



In [2]:
# 1 = Amazonia
# 2 = Caatinga
# 3 = Cerrado
# 4 = Mata Atlantica
# 5 = Pampa
# 6 = Pantanal

# region = "countrywide"
# region = "amaz" #for Amazonia
region = "atl" #for Mata Atlantica
# region = "cerr" #for Cerrado


In [3]:

export_mature_biomass = False
export_distance = False

biomes_br = ee.FeatureCollection(f"{data_folder}/raw/biomes_br")

if region == "countrywide":
    # Countrywide
    region_shp = biomes_br.geometry().dissolve()
elif region == "amaz":
    # Amazonia = 1
    region_shp = biomes_br.filter(ee.Filter.eq("CD_Bioma", 1))
elif region == "cerr":
    region_shp = biomes_br.filter(ee.Filter.eq("CD_Bioma", 3))
elif region == "atl":
    region_shp = biomes_br.filter(ee.Filter.eq("CD_Bioma", 4))



In [5]:
mature_biomass = ee.Image(f"{data_folder}/mature_biomass_500m_countrywide").clip(region_shp)
mature_biomass_vector = mature_biomass.gt(1).reduceToVectors(geometry=region_shp, scale=500, maxPixels=1e13)

secondary = ee.Image(f"{data_folder}/age_agbd").select("age")
distances_image = mature_biomass_vector.distance(searchRadius = 500000).toInt().updateMask(secondary)

def get_nearest_biomass(feature):
    agbd_value = mature_biomass.reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=feature.geometry(),
        scale=500,
        maxPixels=1e13
    )
    return feature.set(agbd_value)

# Function to buffer and calculate mean
def buffer_and_calculate_mean(f):
    distance = ee.Number(f.get('distance'))
    buffered = ee.Feature(f.buffer(distance.add(500), 1))
    mean_value = mature_biomass.reduceRegion(ee.Reducer.mean().unweighted(), \
                                                        buffered.geometry(), 500, maxPixels=1e13)
    return buffered.set(mean_value)

distances_vector = distances_image.sample(region = region_shp, scale=500, numPixels = 10)

# nearest_biomass = distances_vector.map(get_nearest_biomass)
nearest_biomass = distances_vector.map(buffer_and_calculate_mean)

# # Export an ee.FeatureCollection to Google Drive, adjusting the description based on the years
task = ee.batch.Export.table.toAsset(
    collection=nearest_biomass, assetId = f"projects/amazon-forest-regrowth/assets/nearest_biomass_atl", \
          description="near_mat_biomass"
)
task.start()

# map = geemap.Map()
# # # map.addLayer(mature_biomass, {"min": 0, "max":400, 'palette':['blue', 'red']}, "mature_biomass")
# map.addLayer(distances_image, {"min": 0, "max":100000, 'palette':['blue', 'red']}, "dist")
# map.addLayer(nearest_biomass_image, {"min": 0, "max":400, 'palette':['blue', 'red']}, "nearest_biomass")
# map

In [4]:
mature_biomass = ee.Image(f"{data_folder}/mature_biomass_500m_countrywide").clip(region_shp)
mature_biomass_vector = mature_biomass.gt(1).reduceToVectors(geometry=region_shp, scale=500, maxPixels=1e13)
distances = mature_biomass_vector.distance(searchRadius = 500000).toInt()

# Function to buffer and calculate mean
def buffer_and_calculate_mean(f):
    distance = ee.Number(f.get('distance'))
    buffered = ee.Feature(f.buffer(distance.add(500), 1))
    mean_value = mature_biomass.reduceRegion(ee.Reducer.mean().unweighted(), \
                                                        buffered.geometry(), 500, maxPixels=1e13)
    return buffered.set(mean_value)


def get_nearest_biomass(feature):
    agbd_value = mature_biomass.reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=feature.geometry(),
        scale=500,
        maxPixels=1e13
    )
    return feature.set(agbd_value)


# def export_csv(year):
year = 0
if (year == 0):
    land_use_years = ee.Image(f"{data_folder}/land_use")
    name = f"{region}_all"
else:
    land_use_years = ee.Image(f"{data_folder}/land_use_{year}_years")
    name = f"{region}_{year}yr"

age_agbd = ee.Image(f"{data_folder}/age_agbd")
categorical = ee.Image(f"{data_folder}/categorical")
yearly_mean_prec = ee.Image(f"{data_folder}/yearly_mean_prec")
yearly_SI = ee.Image(f"{data_folder}/yearly_SI")
cwd = ee.Image(f"{data_folder}/raw/cwd_chave")
sur_cover = ee.Image(f"{data_folder}/sur_cover").rename("sur_cover")
one_hectare_mask = ee.Image(f"{data_folder}/one_hectare_mask")

combined_mask = one_hectare_mask.And(land_use_years.select("last_LU"))

unified_data = (
    age_agbd.addBands(
        [distances, categorical, land_use_years, cwd, sur_cover, yearly_mean_prec, yearly_SI
    ]).updateMask(combined_mask)
)

unified_data_fc = unified_data.stratifiedSample(
    numPoints = 500, classBand = 'ecoreg', region = unified_data.geometry()
)

pts_bm = unified_data_fc.map(get_nearest_biomass)

task = ee.batch.Export.table.toDrive(
    collection=pts_bm, description=name
)
task.start()

task = ee.batch.Export.table.toAsset(
    collection=pts_bm, description=name, assetId=f"projects/amazon-forest-regrowth/assets/{name}"
)
task.start()

# export_csv(0)
# export_csv(5)
# export_csv(10)
# export_csv(15)